In [1]:
import tabula
import pandas as pd
from tabula import read_pdf

import PyPDF2
import re
# readinf the PDF file that contain Table Data
# you can find find the pdf file with complete code in below
# read_pdf will save the pdf table into Pandas Dataframe

# in order to print first 5 lines of Table



In [3]:
# open the pdf file
pdfFileObject = open('285650.pdf', 'rb')
object = PyPDF2.PdfFileReader(pdfFileObject)

# get number of pages
NumPages = object.getNumPages()

FileNotFoundError: [Errno 2] No such file or directory: '285650.pdf'

In [28]:
NumPages

22

In [31]:
a34 = tabula.read_pdf("285650.pdf", pages=22, multiple_tables=True,guess=False, stream=True)[0]
a34[-15:]

,FORM ADV
996,3. Non-Resident Investment Adviser Undertaking...
997,"By signing this Form ADV, you also agree to pr..."
998,"Regional or District Office of the Commission,..."
999,records that you are required to maintain unde...
1000,"assigns, and any person subject to your writte..."
1001,Signature
1002,"I, the undersigned, sign this Form ADV on beha..."
1003,under the laws of the United States of America...
1004,and that I am signing this Form ADV Execution ...
1005,I certify that the adviser's books and records...


In [32]:
count = 0

f = tabula.read_pdf("285650.pdf", pages=22, multiple_tables=True,guess=False, stream=True)

print(f[0][f[0]['FORM ADV'] == 'A. PRIVATE FUND'])
#count += len(f[0][f[0]['FORM ADV'] == 'A. PRIVATE FUND'])
#A. PRIVATE FUND

Empty DataFrame
Columns: [FORM ADV]
Index: []


In [164]:
count

4

In [147]:
df[0][df[0]['FORM ADV'] == 'A. PRIVATE FUND']

,FORM ADV
711,A. PRIVATE FUND


A sample of the data that we want is in the Attached – Under Section 7.B.(1) Private Fund Reporting…we want question 8, 9, 10 and 11.

In [ ]:
SECTION 7.B.(1) Private Fund Reporting

In [141]:
import pdftotext

# Load your PDF
with open("HighGate.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)

# How many pages?
print(len(pdf))

# Iterate over all the pages
for page in pdf:
    print(page)

# Read some individual pages
print(pdf[0])
print(pdf[1])

# Read all the text into one string
print("\n\n".join(pdf))

ModuleNotFoundError: No module named 'pdftotext'

In [128]:
# import packages
import PyPDF2
import re

# open the pdf file
pdfFileObject = open('HighGate.pdf', 'rb')
object = PyPDF2.PdfFileReader(pdfFileObject)

# get number of pages
NumPages = object.getNumPages()

# define keyterms
String = "Social"

# extract text and do the search
for i in range(0, NumPages):
    PageObj = object.getPage(i)
    print("this is page " + str(i)) 
    Text = PageObj.extractText() 
    # print(Text)
    ResSearch = re.search(String, Text)
    print(ResSearch)

In [ ]:
import PyPDF2
pdfFileObject = open('/Users/guapofish/Desktop/thinkbayes.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
count = pdfReader.numPages
for i in range(count):
    page = pdfReader.getPage(i)
    print(page.extractText())